In [1]:
###############################################################################
"""
Python Glacier Evolution Model "PyGEM" V1.0
Prepared by David Rounce with support from Regine Hock.
This work was funded under the NASA HiMAT project (INSERT PROJECT #).

PyGEM is an open source glacier evolution model written in python.  Model
details come from Radic et al. (2013), Bliss et al. (2014), and Huss and Hock
(2015).
"""
###############################################################################
# This is the main script that provides the architecture and framework for all
# of the model runs. All input data is included in a separate module called
# pygem_input.py. It is recommended to not make any changes to this file unless
# you are a PyGEM developer and making changes to the model architecture.
#
#========== IMPORT PACKAGES ==================================================
# Various packages are used to provide the proper architecture and framework
# for the calculations used in this script. Some packages (e.g., datetime) are
# included in order to speed of calculations and simplify code
import pandas as pd
import numpy as np
from datetime import datetime
import os # os is used with re to find name matches
import re # see os
#========= IMPORT MODEL INPUTS ===============================================
from pygem_input import *
    # import all data
    # pygem_input.py contains all the input data
#========== IMPORT FUNCTIONS FROM MODULES ====================================
import pygemfxns_modelsetup as modelsetup
import pygemfxns_climate as climate
import pygemfxns_massbalance as massbalance

'\nPython Glacier Evolution Model "PyGEM" V1.0\nPrepared by David Rounce with support from Regine Hock.\nThis work was funded under the NASA HiMAT project (INSERT PROJECT #).\n\nPyGEM is an open source glacier evolution model written in python.  Model\ndetails come from Radic et al. (2013), Bliss et al. (2014), and Huss and Hock\n(2015).\n'

In [5]:
#========== OTHER STEPS ======================================================
# Other steps:
# Geodetic mass balance file path
# ???[Create input script full of option list]???
#       - Select regions: (Option 1 - default regions from RGI inventory)
#                         (Option 2 - custom regions)
#       - Select climate data:
#           - climate data source/file name
#           - climate data resolution
#           - sampling scheme (nearest neighbor, etc.)
#       - Model run type: (Option 1 - calibration)
#                         (Option 2 - simulation)
#

#========== LIST OF OUTPUT ===================================================
# Create a list of outputs (.csv or .txt files) that the user can choose from
# depending on what they are using the model for or want to see:
#    1. Time series of all variables for each grid point (x,y,z) of each glacier
#       and every time step
#       (--> comparing with point balances)
#    2. Time series of all variables for elevation bands if spatial
#       discretization is elevation bands (x,y,z) of each glacier and every time
#       step
#       (--> comparing with profiles)
#    3. as above but all variables averaged over current single glacier surfaces
#       (--> comparing with geodetic mass balances of glaciers)
#    4. Time series of all variables averaged over region
#       (--> comparison to GRACE)
#    5. Time series of seasonal balance for individual glaciers
#       (--> comparison to WGMS data)
# Also develop output log file, i.e., file that states input parameters,
# date of model run, model options selected, and any errors that may have come
# up (e.g., precipitation corrected because negative value, etc.)

#========== MODEL RUN DETAILS ================================================
# The model is run through a series of steps:
#   > Step 01: Region/Glaciers Selection
#              The user needs to define the region/glaciers that will be used in
#              the model run.  The user has the option of choosing the standard
#              RGI regions or defining their own.
#   > Step 02: Model Time Frame
#              The user should consider the time step and duration of the model
#              run along with any calibration product and/or model spinup that
#              may be included as well.
#   > Step 03: Climate Data
#              The user has the option to choose the type of climate data being
#              used in the model run, and how that data will be downscaled to
#              the glacier and bins.
#   > Step 04: Glacier Evolution
#              The heart of the model is the glacier evolution, which includes
#              calculating the specific mass balance, the surface type, and any
#              changes to the size of the glacier (glacier dynamics). The user
#              has many options for how this aspect of the model is run.
#   > Others: model output? model input?

#========== LIST OF MODEL VARIABLES (alphabetical) ===========================
# Prefixes and Suffixes:
#   _annual: dataframe containing information with an annual time step as
#           opposed to the time step specified by the model (daily or monthly).
#   _bin_:  dataframe containing information related to each elevation bin/band
#           on the glacier. These dataframes are indexed such that the main
#           index (rows) are elevation bins and the columns are the time series.
#   _glac_: dataframe containing information related to a glacier. When used by
#           itself (e.g., main_glac_rgi or gcm_glac_temp) it refers to each row
#           being a specific glacier. When used as a prefix followed by a
#           descriptor (e.g., glac_bin_temp), the entire dataframe is for one
#           glacier and the descriptor provides information as to the rows
#   gcm_:   meteorological data from the global climate model or reanalysis
#           dataset
#   main_:  dataframe containing important information related to all the
#           glaciers in the study, where each row represents a glacier (ex.
#           main_glac_rgi).
#   series_: series containing information for a given time step with respect to
#           all the glacier bins. This is needed when cycling through each
#           time step to calculate the mass balance since snow accumulates and
#           alters the surface type.
#
# Variables:
#   dates_table: dataframe of the dates, month, year, and number of days in the
#           month for each date.
#           Rows = dates, Cols = attributes
# > end_date: end date of model run
#           (MAY NOT BE NEEDED - CHECK WHERE IT'S USED)
#   gcm_glac_elev: Series of elevation data associated with the global climate
#           model temperature data
#   gcm_glac_prec: dataframe of the global climate model precipitation data,
#           typically based on the nearest neighbor.
#   gcm_glac_temp: dataframe of the global climate model temperature data,
#           typically based on the nearest neighbor.
#   glac_bin_temp: dataframe of temperature for each bin for each time step on
#           the glacier
#   glac_bin_prec: dataframe of precipitation (liquid) for each bin for each
#           time step on the glacier
#   glac_bin_precsnow: dataframe of the total precipitation (liquid and solid)
#           for each bin for each time step on the glacier
#   glac_bin_snow: dataframe of snow for each bin for each time step on
#           the glacier
#   glac_bin_snowonsurface: dataframe of the snow that has accumulated on the
#           surface of the glacier for each bin for each time step
#   glac_bin_surftype: datframe of the surface type for each bin for each time
#           step on the glacier
#   main_glac_rgi: dataframe containing generic attributes (RGIId, Area, etc.)
#           from the Randolph Glacier Inventory for each glacier.
#           Rows = glaciers, Cols = attributes
#   main_glac_parameters: dataframe containing the calibrated parameters for
#           each glacier
#   main_glac_surftypeinit: dataframe containing in the initial surface type for
#           each glacier
# > start_date: start date of model run
#           (MAY NOT BE NEEDED - CHECK WHERE IT'S USED)

In [6]:
#----- STEP ONE: MODEL REGION/GLACIERS ---------------------------------------
# Step one involves the selection of the regions and glaciers used in the model.
# Regions/glacier included in the model run will be defined using the Randolph
#   Glacier inventory.  For more information, see:
#     https://www.glims.org/RGI/ (RGI Consortium, 2017)
# In step one, the model will:
#   > select glaciers included in model run

# Select glaciers that are included in the model run
# Glacier Selection Options:
#   > 1 (default) - enter numbers associated with RGI V6.0 and select
#                   glaciers accordingly
#   > 2 - glaciers/regions selected via shapefile
#   > 3 - glaciers/regions selected via new table (other inventory)
#
if option_glacier_selection == 1:
    main_glac_rgi = modelsetup.selectglaciersrgitable()
elif option_glacier_selection == 2:
    # OPTION 2: CUSTOMIZE REGIONS USING A SHAPEFILE that specifies the
    #           various regions according to the RGI IDs, i.e., add an
    #           additional column to the RGI table.
    # ??? [INSERT CODE FOR IMPORTING A SHAPEFILE] ???
    #   (1) import shapefile with custom boundaries, (2) grab the RGIIDs
    #   of glaciers that are in these boundaries, (3) perform calibration
    #   using these alternative boundaries that may (or may not) be more
    #   representative of regional processes/climate
    #   Note: this is really only important for calibration purposes and
    #         post-processing when you want to show results over specific
    #         regions.
    # Development Note: if create another method for selecting glaciers,
    #                   make sure that update way to select glacier
    #                   hypsometry as well.
    print('\n\tMODEL ERROR (selectglaciersrgi): this option to use'
          '\n\tshapefiles to select glaciers has not been coded yet.'
          '\n\tPlease choose an option that exists. Exiting model run.\n')
    exit()
else:
    # Should add options to make regions consistent with Brun et al. (2017),
    # which used ASTER DEMs to get mass balance of 92% of the HMA glaciers.
    print('\n\tModel Error (selectglaciersrgi): please choose an option'
          '\n\tthat exists for selecting glaciers. Exiting model run.\n')
    exit()


This study is only focusing on glaciers ['05000', '07743'] in region [13].
The 'select_rgi_glaciers' function has finished.


In [7]:
#----- STEP TWO: ADDITIONAL MODEL SETUP --------------------------------------
# Step two runs more functions related to the model setup. This section has been
#   separated from the selection of the model region/glaciers in order to
#   keep the input organized and easy to read.
# In step two, the model will:
#   > select glacier hypsometry
#   > define the model time frame
#   > define the initial surface type

# Glacier hypsometry
# main_glac_hyps = modelsetup.hypsometryglaciers(main_glac_rgi)
    # Note: need to adjust this hypsometry into separate functions such that it
    #       is easier to follow.
# AUTOMATE THIS TO LOAD THEM IN INSTEAD OF CHOOSING THEM
main_glac_hyps = pd.read_csv(hyps_filepath + 'RGI_13_area_test20170905.csv')

# Glacier initial ice thickness
# AUTOMATE THIS TO LOAD THEM IN INSTEAD OF CHOOSING THEM
main_glac_icethickness = pd.read_csv(hyps_filepath +
                                     'RGI_13_thickness_test20170905.csv')
# ADD OPTION FOR VOLUME-AREA SCALING

# Glacier total initial volume
main_glac_rgi['Volume'] = (
    (main_glac_hyps * main_glac_icethickness/1000).sum(axis=1))
    # volume [km3] = area[km2] * thickness[m] * (1 [km] / 1000 [m])

print(main_glac_rgi.head(),'\n')


# Model time frame
#   Set up table of dates. These dates are used as column headers for many other
#   variables in the model run, so it's important to be an initial step.
dates_table, start_date, end_date = modelsetup.datesmodelrun(option_leapyear)

# Initial surface type
main_glac_surftypeinit = modelsetup.surfacetypeglacinitial(
                                                option_surfacetype_initial,
                                                option_surfacetype_firn,
                                                option_surfacetype_debris,
                                                main_glac_rgi,
                                                main_glac_hyps)


        O1Index           RGIId     CenLon     CenLat  O1Region  O2Region  \
GlacNo                                                                      
0          4999  RGI60-13.05000  80.193268  42.062546        13         3   
1          7742  RGI60-13.07743  78.449058  42.152977        13         3   

           Area  Zmin  Zmax  Zmed  Slope  Aspect   Lmax     Volume  
GlacNo                                                              
0       373.922  2901  6643  4475   13.3     304  61445  84.527037  
1         6.364  3488  5014  4420   15.0     178   6870   0.533249   

The 'datesmodelrun' function has finished.
The 'initialsurfacetype' function has finished.


In [ ]:
#----- STEP THREE: CLIMATE DATA ----------------------------------------------
# Step three imports the climate data that will be used in the model run.
# Provide options for the selection and downscaling of the data
#    - default: nearest neighbor
#    - alternatives: weighted methods
#      (note: prior to any weighting, lapse rates/biases need to be applied)
# Important to consider the following:
#    - time period of the calibration data or model run
#    - type of model (DDF, EBM, etc.) will dictate meteorological data needed
#   Datasets:
#     Past: Default: ERA reanslysis?
#           Alternatives: COAWST (S.Nichols), NCEP/NCAR(?), others?
#                         automatic weather stations
#     Future: Default: GCMs (see glacierMIP project emails to download data)
#             Alternatives: COAWST (S.Nichols), others?
#
# In step three, the model will:
#   > import meteorological data
#   > select meteorological data for each glacier based on specified option
#       default: nearest neighbor
if option_gcm_downscale == 1:
    # OPTION 1 (default): NEAREST NEIGHBOR
    # Thoughts on 2017/08/21:
    #   > Pre-processing functions should be coded and added after the initial
    #     import such that the initial values can be printed if necessary.
    #   > Data imported here is monthly, i.e., it is 1 value per month. If the
    #     data is going to be subsampled to a daily resolution in order to
    #     estimate melt in areas with low monthly mean temperature as is done in
    #     Huss and Hock (2015), then those calculations should be performed in
    #     the ablation section.
    gcm_glac_temp = climate.importGCMvarnearestneighbor(gcm_temp_varname,
                                                        main_glac_rgi,
                                                        dates_table)
        # gcm nearest neighbor time series for each glacier with GlacNo index
        # rows = # of glaciers, cols = length of time series
    gcm_glac_prec = climate.importGCMvarnearestneighbor(gcm_prec_varname,
                                                        main_glac_rgi,
                                                        dates_table)
        # gcm nearest neighbor time series for each glacier with GlacNo index
        # rows = # of glaciers, cols = length of time series
    gcm_glac_elev = climate.importGCMfxnearestneighbor(gcm_elev_varname,
                                                       main_glac_rgi)
        # gcm nearest neighbor surface altitude for each glacier with GlacNo
        # index, rows = # of glaciers, cols = 1 (Series)
else:
    print('\n\tModel Error: please choose an option that exists for'
          '\n\tdownscaling climate data. Exiting model run now.\n')
    exit() # if you have an error, exit the model run
